# LSTM 

## Outline
1. [Imports](#imports)
2. [Data](#data)
3. [Model](#model)
5. [Train](#train)
6. [Predict](#predict)
7. [Evaluate](#evaluate)
7. [Metrics](#metrics)
8. [Plots](#plots)

<a name=imports></a>
## Imports

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

import os
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

import preprocessing

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

***
<a name=data></a>
## Data

### Ingestion
Load in the dataset as pandas dataframe.

In [ ]:
# Check preprocessing.py for implementation
(india_cases_df, india_vacc_df, usa_cases_df, usa_vacc_df) = preprocessing.ingestion()

### Pre-Processing
Filter out unnecessary columns. 

#### Multivariate

In [ ]:
# Check preprocessing.py for implementation
# Number of Null in India - 563
# Number of Null in USA - 261
(india_multi, usa_multi) = preprocessing.multivariate(india_cases_df, india_vacc_df, usa_cases_df, usa_vacc_df)

#### Univariate 

In [ ]:
# Check preprocessing.py for implementation
# Number of Null in India - 0
# Number of Null in USA - 0
(india_cases_uni, usa_cases_uni) = preprocessing.univariate(india_cases_df, usa_cases_df)

### Drop Null
Merging the vaccination and #cases datasets together for multivariate time series produces null values. Dropping the nuWe solve this by dropping the null values.ll values ensures that we take an inner join.

In [ ]:
(india_multi, usa_multi) = preprocessing.dropNull(india_multi, usa_multi)

### Normalize
Perform a Z-score normalization.

In [ ]:
# Check preprocessing.py for implementation
(india_uni_norm, usa_uni_norm, india_multi_norm, usa_multi_norm,india_cases_mean,india_cases_std,usa_cases_mean,usa_cases_std,india_multi_mean,india_multi_std,
usa_multi_mean, usa_multi_std) = preprocessing.normalize(india_cases_uni, 
                                                                                           usa_cases_uni, 
                                                                                           india_multi, 
                                                                                           usa_multi)

### Create Splits
Break the dataset into train and test. The current split is 80:20.

In [ ]:
# Create train test splits
(india_uni_train, india_uni_test, usa_uni_train, usa_uni_test) = preprocessing.split(india_uni_norm, usa_uni_norm, 0.2)
(india_multi_train, india_multi_test, 
 usa_multi_train, usa_multi_test) = preprocessing.split(india_multi_norm, usa_multi_norm, 0.2)

### Data Transformation
Simple conversion from pandas dataframe to numpy array by extracting the values.

In [ ]:
# Univariate
india_uni_train_np, india_uni_test_np = india_uni_train.values,  india_uni_test.values
usa_uni_train_np, usa_uni_test_np = usa_uni_train.values, usa_uni_test.values
# Multivariate
india_multi_train_np, india_multi_test_np = india_multi_train.values,  india_multi_test.values
usa_multi_train_np, usa_multi_test_np = usa_multi_train.values, usa_multi_test.values

In [ ]:
print(india_uni_train_np.shape, india_uni_test_np.shape)
print(india_multi_train_np.shape, india_multi_test_np.shape)

### Reshape
Converts the exiting train data set to match the input parameters of an LSTM (Samples, Time Steps, Features). The `lstm_data_transform()` method takes in 3 arguments: X data, y data and number of features.

We will be performing autoregression on the univariate data, therefore the X and y values are the same. As for the multivariate data, we will be using the confirmed column (index 0) as the y value.

In [ ]:
india_uni_train_np

In [ ]:
timesteps = 14

In [ ]:
# Univariate
india_uni_train_X, india_uni_train_y = preprocessing.lstm_data_transform(india_uni_train_np, india_uni_train_np,
                                                                         timesteps)
usa_uni_train_X, usa_uni_train_y = preprocessing.lstm_data_transform(usa_uni_train_np, usa_uni_train_np,
                                                                     timesteps)
# Multivariate
india_multi_train_X, india_multi_train_y = preprocessing.lstm_data_transform(india_multi_train_np, india_multi_train_np, timesteps)
usa_multi_train_X, usa_multi_train_y = preprocessing.lstm_data_transform(usa_multi_train_np, usa_multi_train_np, timesteps)

In [ ]:
print("Univariate India shape:",india_uni_train_X.shape, india_uni_train_y.shape)
print("Multivariate India shape:",india_multi_train_X.shape, india_multi_train_y.shape)

The univariate array shape for the India dataset has 566 samples, 3 as the time step, and 1 feature for the X values. The y values is just the 566 samples.

Similarly, the multivariate array shape for the India dataset has 276 samples, 3 as the time step, and 2 features for the X values. The y values is just the 276 samples of the 0th column (Confirmed cases).

***
<a name=model></a>
## Model
We will define 4 models in total. One for the univariate data and another for the multivariate for each country.

In [ ]:
import LSTMmodels
from keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf

In [ ]:
univariate_model_ind = LSTMmodels.build_lstm(time_steps=india_uni_train_X.shape[1],
                                             features=india_uni_train_X.shape[2])
univariate_model_usa = LSTMmodels.build_lstm(time_steps=usa_uni_train_X.shape[1],
                                             features=usa_uni_train_X.shape[2])

In [ ]:
multivariate_model_ind = LSTMmodels.build_lstm(time_steps=india_multi_train_X.shape[1],
                                               features=india_multi_train_X.shape[2], outputs=2)
multivariate_model_usa = LSTMmodels.build_lstm(time_steps=usa_multi_train_X.shape[1],
                                               features=usa_multi_train_X.shape[2], outputs=2)

***
<a name=train></a>
## Train 

In [ ]:
# Get the path of the current working directory
curPath = os.getcwd()
# Appened the parent directory to the current path to step out of the current folder
parentDir = os.path.abspath(os.path.join(curPath, os.pardir))
# Tensorboard callback
logdir = os.path.join(parentDir+"/logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
def train_models(model,train_X,train_y,test_X,test_y):
    model.fit(train_X,train_y,
              validation_data=(test_X,test_y),
              epochs=300,
              callbacks=[tensorboard_callback])
    return model
#     model.save(save_name)

In [ ]:
# Train the model on univariate India dataset 
univariate_model_ind = LSTMmodels.train_model(univariate_model_ind,india_uni_train_X,india_uni_train_y,
                                              tensorboard_callback)

In [ ]:
# Train the model on univariate USA dataset 
univariate_model_usa = LSTMmodels.train_model(univariate_model_usa,usa_uni_train_X,usa_uni_train_y,
                                              tensorboard_callback)

In [ ]:
# Train the model on multivariate India dataset
multivariate_model_ind = LSTMmodels.train_model(multivariate_model_ind,india_multi_train_X,india_multi_train_y,
                                                tensorboard_callback)

In [ ]:
# Train the model on multivariate USA dataset
multivariate_model_usa = LSTMmodels.train_model(multivariate_model_usa,usa_multi_train_X,usa_multi_train_y,
                                                tensorboard_callback)

***
<a name=predict></a>
## Predict
There are 2 different types of prediction methods. 
1. Long-term predictions

This type of prediction uses a sliding window to continue to predict into the future by continuing to use the forecasted values.

2. Rolling short-term predictions

This prediction method continues to train the LSTM model every 14 iterations. Unlike the long-term predictions, this model forecasts one day at a time.

In [ ]:
import evaluate

#### Long-Term 

In [ ]:
# Univariate Long-Term
yhat_uni_india = evaluate.make_predictions(univariate_model_ind,india_uni_test_np,timesteps,india_uni_test_np.shape[1])
yhat_uni_usa = evaluate.make_predictions(univariate_model_usa,usa_uni_test_np,timesteps,usa_uni_test_np.shape[1])

In [ ]:
# Multivariate Long-Term 
yhat_multi_india_long = evaluate.make_predictions(multivariate_model_ind,india_multi_test_np,timesteps,india_multi_test_np.shape[1])
yhat_multi_usa_long = evaluate.make_predictions(multivariate_model_usa,usa_multi_test_np,timesteps,usa_multi_test_np.shape[1])

#### Rolling Short-Term 

In [ ]:
# Univariate Rolling Short-Term
yhat_uni_india_short, y_uni_india_short, univariate_model_ind = evaluate.make_predictions_short(univariate_model_ind,
                                                                                                india_uni_train_np,
                                                                                                india_uni_test_np,timesteps,
                                                                                                india_uni_test_np.shape[1],
                                                                                               tensorboard_callback)
yhat_uni_usa_short, y_uni_usa_short, univariate_model_usa = evaluate.make_predictions_short(univariate_model_usa, 
                                                                                            usa_uni_train_np,
                                                                                            usa_uni_test_np,timesteps,
                                                                                            usa_uni_test_np.shape[1],
                                                                                           tensorboard_callback)

In [ ]:
# Multivariate Rolling Short-Term
yhat_multi_india, y_multi_india_short, multivariate_model_ind = evaluate.make_predictions_short(multivariate_model_ind, india_multi_train_np,india_multi_test_np,timesteps,
                                                                                                india_multi_test_np.shape[1],
                                                                                               tensorboard_callback, india_multi_test_np.shape[0])
yhat_multi_usa, y_multi_india_short, multivariate_model_usa = evaluate.make_predictions_short(multivariate_model_usa,usa_multi_train_np,usa_multi_test_np,
                                                                      timesteps,usa_multi_test_np.shape[1],
                                                                     tensorboard_callback, usa_multi_test_np.shape[0])

In [ ]:
# print(yhat_multi_india)
# print(yhat_multi_usa.shape)
yhat_multi_india = np.concatenate(yhat_multi_india).flatten()
print(yhat_multi_india)
yhat_multi_usa = np.concatenate(yhat_multi_usa).flatten()
print(yhat_multi_usa)

# yhat_multi_india = yhat_multi_india[0]
# yhat_multi_usa = yhat_multi_usa[0]
yhat_multi_india_long = yhat_multi_india_long[0]
yhat_multi_usa_long = yhat_multi_usa_long[0]

In [ ]:
print(yhat_multi_india.shape,
yhat_multi_usa.shape,
yhat_multi_india_long.shape,
yhat_multi_usa_long.shape)

***
<a name=evaluate></a>
## Evaluate

### Denormalize Data
First step is to denormalize the data back to it's original scale.

#### Long-Term 

##### Univariate 

In [ ]:
# Denormalize univariate India data
india_uni_test_np_denorm = preprocessing.de_normalize(india_uni_test_np.flatten()[timesteps:],np.array(india_cases_mean),np.array(india_cases_std))
yhat_uni_india_denorm = preprocessing.de_normalize(yhat_uni_india.flatten()[timesteps:],np.array(india_cases_mean),np.array(india_cases_std))

# Round values to avoid floating point errors
india_uni_test_np_long_denorm = np.round(india_uni_test_np_denorm)
yhat_uni_india_long_denorm = np.round(yhat_uni_india_denorm)

In [ ]:
# Denormalize univariate USA data
usa_uni_test_np_denorm = preprocessing.de_normalize(usa_uni_test_np.flatten()[timesteps:],np.array(usa_cases_mean),np.array(usa_cases_std))
yhat_uni_usa_denorm = preprocessing.de_normalize(yhat_uni_usa.flatten()[timesteps:],np.array(usa_cases_mean),np.array(usa_cases_std))

# Round values to avoid floating point errors
usa_uni_test_np_long_denorm = np.round(usa_uni_test_np_denorm)
yhat_uni_usa_long_denorm = np.round(yhat_uni_usa_denorm)

##### Multivariate

In [ ]:
# Denormalize multivariate India data - long term
india_multi_test_np_long_denorm = preprocessing.de_normalize(india_multi_test_np[:,0].flatten()[timesteps:],np.array(india_multi_mean[0]),np.array(india_multi_std[0]))
yhat_multi_india_long_denorm = preprocessing.de_normalize(yhat_multi_india_long[:,0].flatten()[0::2][timesteps:],np.array(india_multi_mean[0]),np.array(india_multi_std[0]))

# Round values to avoid floating point errors
india_multi_test_np_long_denorm = np.round(india_multi_test_np_long_denorm)
yhat_multi_india_long_denorm = np.round(yhat_multi_india_long_denorm)

In [ ]:
# Denormalize multivariate USA data - long term
usa_multi_test_np_long_denorm = preprocessing.de_normalize(usa_multi_test_np[:,0].flatten()[timesteps:],np.array(usa_multi_mean[0]),np.array(usa_multi_std[0]))
yhat_multi_usa_long_denorm = preprocessing.de_normalize(yhat_multi_usa_long[:,0].flatten()[0::2][timesteps:],np.array(usa_multi_mean[0]),np.array(usa_multi_std[0]))

# Round values to avoid floating point errors
usa_multi_test_np_long_denorm = np.round(usa_multi_test_np_long_denorm)
yhat_multi_usa_long_denorm = np.round(yhat_multi_usa_long_denorm)

#### Rolling Short-Term 

In [ ]:
yhat_multi_india_long[0]

##### Univariate 

In [ ]:
# Reshaping the data to standard dim 1
yhat_uni_india_short = np.concatenate(yhat_uni_india_short).flatten()

# Denormalize univariate short India data
india_uni_test_short_denorm = preprocessing.de_normalize(y_uni_india_short,np.array(india_cases_mean),np.array(india_cases_std))
yhat_uni_india_short_denorm = preprocessing.de_normalize(yhat_uni_india_short,np.array(india_cases_mean),np.array(india_cases_std))

# Round values to avoid floating point errors
india_uni_test_short_denorm = np.round(india_uni_test_short_denorm)
yhat_uni_india_short_denorm = np.round(yhat_uni_india_short_denorm)

In [ ]:
# Reshaping the data to standard dim 1
yhat_uni_usa_short = np.concatenate(yhat_uni_usa_short).flatten()

# Denormalize univariate short USA data
usa_uni_test_short_denorm = preprocessing.de_normalize(y_uni_usa_short,np.array(usa_cases_mean),np.array(usa_cases_std))
yhat_uni_usa_short_denorm = preprocessing.de_normalize(yhat_uni_usa_short,np.array(usa_cases_mean),np.array(usa_cases_std))

# Round values to avoid floating point errors
usa_uni_test_short_denorm = np.round(usa_uni_test_short_denorm)
yhat_uni_usa_short_denorm = np.round(yhat_uni_usa_short_denorm)

##### Multivariate

In [ ]:
# Denormalize multivariate India data - short term
india_multi_test_np_denorm = preprocessing.de_normalize(india_multi_test_np[:,0].flatten(),np.array(india_multi_mean[0]),np.array(india_multi_std[0]))
yhat_multi_india_denorm = preprocessing.de_normalize(yhat_multi_india.flatten(),np.array(india_multi_mean[0]),np.array(india_multi_std[0]))

# Round values to avoid floating point errors
india_multi_test_np_short_denorm = np.round(india_multi_test_np_denorm)
yhat_multi_india_short_denorm = np.round(yhat_multi_india_denorm)[0::2]

In [ ]:
# Denormalize multivariate USA data - short term
usa_multi_test_np_denorm = preprocessing.de_normalize(usa_multi_test_np[:,0].flatten(),np.array(usa_multi_mean[0]),np.array(usa_multi_std[0]))
yhat_multi_usa_denorm = preprocessing.de_normalize(yhat_multi_usa.flatten(),np.array(usa_multi_mean[0]),np.array(usa_multi_std[0]))

# Round values to avoid floating point errors
usa_multi_test_np_short_denorm = np.round(usa_multi_test_np_denorm)
yhat_multi_usa_short_denorm = np.round(yhat_multi_usa_denorm)[0::2]

***
<a name=metrics></a>
## Get Metrics 
Return format: (mape, mae, rmse)

#### Long-Term

In [ ]:
# Check to ensure data matches
denorm_test = evaluate.evaluate(india_cases_uni.values[-(india_uni_test_short_denorm.size):].flatten(),india_uni_test_short_denorm)
denorm_test

##### Univariate

In [ ]:
# Univariate Long-Term India
uni_india_eval_long = evaluate.evaluate(india_uni_test_np_long_denorm, yhat_uni_india_long_denorm)
uni_india_eval_long.insert(0,'Univariate Long-Term India')
uni_india_eval_long

In [ ]:
# Univariate Long-Term USA
uni_usa_eval_long = evaluate.evaluate(usa_uni_test_np_long_denorm, yhat_uni_usa_long_denorm)
uni_usa_eval_long.insert(0,'Univariate Long-Term USA')
uni_usa_eval_long

##### Multivariate

In [ ]:
# Multivariate Long-Term India
multi_india_eval_long = evaluate.evaluate(india_multi_test_np_long_denorm, yhat_multi_india_long_denorm)
multi_india_eval_long.insert(0,'Multivariate Long-Term India')
multi_india_eval_long

In [ ]:
# Multivariate Long-Term USA
multi_usa_eval_long = evaluate.evaluate(usa_multi_test_np_long_denorm, yhat_multi_usa_long_denorm)
multi_usa_eval_long.insert(0,'Multivariate Long-Term USA')
multi_usa_eval_long

#### Short-Term Rolling Forecast

##### Univariate 

In [ ]:
# Univariate Short-Term India
uni_india_eval_short = evaluate.evaluate(india_uni_test_short_denorm, yhat_uni_india_short_denorm)
uni_india_eval_short.insert(0,'Univariate Short-Term India')
uni_india_eval_short

In [ ]:
# Univariate Short-Term USA
uni_usa_eval_short = evaluate.evaluate(usa_uni_test_short_denorm, yhat_uni_usa_short_denorm)
uni_usa_eval_short.insert(0,'Univariate Short-Term USA')
uni_usa_eval_short

##### Multivariate

In [ ]:
# Multivariate Short-Term India
multi_india_eval_short = evaluate.evaluate(india_multi_test_np_short_denorm, yhat_multi_india_short_denorm)
multi_india_eval_short.insert(0,'Multivariate Short-Term India')
multi_india_eval_short

In [ ]:
# Multivariate Short-Term USA
multi_usa_eval_short = evaluate.evaluate(usa_multi_test_np_short_denorm, yhat_multi_usa_short_denorm)
multi_usa_eval_short.insert(0,'Multivariate Short-Term USA')
multi_usa_eval_short

### Visualize performance
TODO: Still need to add multivariate metrics

In [ ]:
d = [uni_india_eval_long,uni_usa_eval_long,uni_india_eval_short,uni_usa_eval_short,multi_india_eval_long,
    multi_usa_eval_long,multi_india_eval_short,multi_usa_eval_short]
metrics_table = pd.DataFrame(data=d, columns=['Models','MAPE','MAE','RMSE'])
metrics_table

***
<a name=plots></a>
## Plots
March 1st - 

In [ ]:
%tensorboard --logdir logdir

#### Long-Term Rolling forecasts

##### Univariate

In [ ]:
evaluate.plot_fore_test(india_uni_test_np_long_denorm, yhat_uni_india_long_denorm, 'Univariate Long-Term India')

In [ ]:
evaluate.plot_fore_test(usa_uni_test_np_long_denorm, yhat_uni_usa_long_denorm, 'Univariate Long-Term USA')

##### Multivariate

In [ ]:
evaluate.plot_fore_test(india_multi_test_np_long_denorm, yhat_multi_india_long_denorm, "Multivariate Long-Term India")

In [ ]:
evaluate.plot_fore_test(usa_multi_test_np_long_denorm, yhat_multi_usa_long_denorm, "Multivariate Long-Term USA")

#### Short-Term Rolling Forecast

##### Univariate

In [ ]:
evaluate.plot_fore_test(india_uni_test_short_denorm, yhat_uni_india_short_denorm, "Univariate Short-Term India")

In [ ]:
evaluate.plot_fore_test(usa_uni_test_short_denorm, yhat_uni_usa_short_denorm, "Univariate Short-Term USA")

##### Multivariate

In [ ]:
evaluate.plot_fore_test(india_multi_test_np_short_denorm, yhat_multi_india_short_denorm, "Multivariate Short-Term India")

In [ ]:
evaluate.plot_fore_test(usa_multi_test_np_short_denorm, yhat_multi_usa_short_denorm, "Multivariate Short-Term USA")

***
## Save Results
Split = 80:20
India has 568 train, 143 test
Usa has 511 train, 143 test

Long-term predictions have 14 less days in test

In [ ]:
# Get the path of the current working directory
curPath = os.getcwd()
# Appened the parent directory to the current path to step out of the current folder
parentDir = os.path.abspath(os.path.join(curPath, os.pardir))
print("Parent Directory", parentDir)
figpath = os.path.join(parentDir,"../figures/lstm")

In [ ]:
import plotting

In [ ]:
dates = india_cases_df.values[1:,0]
dates_df = pd.to_datetime(dates,format='%Y-%m-%d')
dates_df[-129:]

In [ ]:
# Train values India
india_train_df = india_cases_df[['Date','Confirmed']][1:]
india_train_df['Date'] = pd.to_datetime(india_train_df['Date'],format='%Y-%m-%d')
india_train_df.set_index(['Date'],inplace=True)
india_train_df.loc['2021-03-01':]

In [ ]:
# Train values USA
usa_train_df = usa_cases_df[['Date','Confirmed']][1:]
usa_train_df['Date'] = pd.to_datetime(usa_train_df['Date'],format='%Y-%m-%d')
usa_train_df.set_index(['Date'],inplace=True)
usa_train_df

#### Long-Term

In [ ]:
# Univariate India

# Y values
y_india_uni_long_df = pd.DataFrame({'y_india_uni_long':india_uni_test_np_long_denorm})
y_india_uni_long_df.set_index(dates_df[-129:],inplace=True)
print(y_india_uni_long_df.head())

# yhat values
yhat_uni_india_long_df = pd.DataFrame({'yhat_india_uni_long':yhat_uni_india_long_denorm})
yhat_uni_india_long_df.set_index(dates_df[-129:],inplace=True)
print(yhat_uni_india_long_df.head())

In [ ]:
plotting.plot_train_test_fore(train=india_train_df,test=y_india_uni_long_df,fore=yhat_uni_india_long_df,
                              title='India - Long Term',figpath=figpath+'/india_unilstm.eps')

In [ ]:
# Univariate USA

# Y values
y_usa_uni_long_df = pd.DataFrame({'y_usa_uni_long':usa_uni_test_np_long_denorm})
y_usa_uni_long_df.set_index(dates_df[-114:],inplace=True)
print(y_usa_uni_long_df.head())

# yhat values
yhat_uni_usa_long_df = pd.DataFrame({'yhat_usa_uni_long':yhat_uni_usa_long_denorm})
yhat_uni_usa_long_df.set_index(dates_df[-114:],inplace=True)
print(yhat_uni_usa_long_df.head())

In [ ]:
plotting.plot_train_test_fore(train=usa_train_df,test=y_usa_uni_long_df,fore=yhat_uni_usa_long_df,
                              title='USA - Long Term',figpath=figpath+'/usa_unilstm.eps')

In [ ]:
# Multivariate India

# Y values
y_india_multi_long_df = pd.DataFrame({'y_india_multi_long':india_multi_test_np_long_denorm})
y_india_multi_long_df.set_index(dates_df[-129:],inplace=True)
print(y_india_multi_long_df.head())

# yhat values
yhat_multi_india_long_df = pd.DataFrame({'yhat_india_multi_long':yhat_multi_india_long_denorm})
yhat_multi_india_long_df.set_index(dates_df[-129:],inplace=True)
print(yhat_multi_india_long_df.head())

In [ ]:
plotting.plot_train_test_fore(train=india_train_df,test=y_india_multi_long_df,fore=yhat_multi_india_long_df,
                              title='India - Long Term',figpath=figpath+'/india_multilstm.eps')

In [ ]:
# Multivariate USA

# Y values
y_usa_multi_long_df = pd.DataFrame({'y_usa_multi_long':usa_multi_test_np_long_denorm})
y_usa_multi_long_df.set_index(dates_df[-114:],inplace=True)
print(y_usa_multi_long_df.head())

# yhat values
yhat_multi_usa_long_df = pd.DataFrame({'yhat_usa_multi_long':yhat_multi_usa_long_denorm})
yhat_multi_usa_long_df.set_index(dates_df[-114:],inplace=True)
print(yhat_multi_usa_long_df.head())

In [ ]:
plotting.plot_train_test_fore(train=usa_train_df,test=y_usa_multi_long_df,fore=yhat_multi_usa_long_df,
                              title='USA - Long Term',figpath=figpath+'/usa_multilstm.eps')

#### Short-Term 

In [ ]:
# Univariate India

# Y values
y_india_uni_short_df = pd.DataFrame({'y_india_uni_short':india_uni_test_short_denorm})
y_india_uni_short_df.set_index(dates_df[-143:],inplace=True)
print(y_india_uni_long_df.head())

# yhat values
yhat_uni_india_short_df = pd.DataFrame({'yhat_india_uni_short':yhat_uni_india_short_denorm})
yhat_uni_india_short_df.set_index(dates_df[-143:],inplace=True)
print(yhat_uni_india_short_df.head())

In [ ]:
plotting.plot_train_test_fore(train=india_train_df,test=y_india_uni_short_df,fore=yhat_uni_india_short_df,
                              title='India - Short Term',figpath=figpath+'/india_unilstm_rolling.eps')

In [ ]:
# Univariate USA

# Y values
y_usa_uni_short_df = pd.DataFrame({'y_usa_uni_short':usa_uni_test_short_denorm})
y_usa_uni_short_df.set_index(dates_df[-128:],inplace=True)
print(y_usa_uni_short_df.head())

# yhat values
yhat_uni_usa_short_df = pd.DataFrame({'yhat_usa_uni_short':yhat_uni_usa_short_denorm})
yhat_uni_usa_short_df.set_index(dates_df[-128:],inplace=True)
print(yhat_uni_usa_short_df.head())

In [ ]:
plotting.plot_train_test_fore(train=usa_train_df,test=y_usa_uni_short_df,fore=yhat_uni_usa_short_df,
                              title='USA - Short Term',figpath=figpath+'/usa_unilstm_rolling.eps')

In [ ]:
# Multivariate India

# Y values
y_india_multi_short_df = pd.DataFrame({'y_india_multi_short':india_multi_test_np_short_denorm})
y_india_multi_short_df.set_index(dates_df[-143:],inplace=True)
print(y_india_multi_short_df.head())

# yhat values
yhat_multi_india_long_df = pd.DataFrame({'yhat_india_multi_short':yhat_multi_india_short_denorm})
yhat_multi_india_long_df.set_index(dates_df[-143:],inplace=True)
print(yhat_multi_india_long_df.head())

In [ ]:
plotting.plot_train_test_fore(train=india_train_df,test=y_india_multi_short_df,fore=yhat_multi_india_long_df,
                              title='India - Short Term',figpath=figpath+'/india_multilstm_rolling.eps')

In [ ]:
# Multivariate USA

# Y values
y_usa_multi_short_df = pd.DataFrame({'y_usa_multi_short':usa_multi_test_np_short_denorm})
y_usa_multi_short_df.set_index(dates_df[-128:],inplace=True)
print(y_usa_multi_short_df.head())

# yhat values
yhat_multi_usa_short_df = pd.DataFrame({'yhat_usa_multi_short':yhat_multi_usa_short_denorm})
yhat_multi_usa_short_df.set_index(dates_df[-128:],inplace=True)
print(yhat_multi_usa_short_df.head())

In [ ]:
plotting.plot_train_test_fore(train=usa_train_df,test=y_usa_multi_short_df,fore=yhat_multi_usa_short_df,
                              title='USA - Short Term',figpath=figpath+'/usa_multilstm_rolling.eps')